In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl 
mpl.rc('font',family='Malgun Gothic')
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats as stats

In [2]:
df_I10= pd.read_csv('../../datasets/Join_Collection_NSC2_BND_M20.csv')
df_I10.columns

Index(['Unnamed: 0', 'EDC_SBA', 'OPRTN_YN', 'MDCARE_DD_CNT', 'BTH_YYYY',
       'MCARE_RSLT_TYPE', 'VSHSP_DD_CNT', 'EDC_ADD_RT'],
      dtype='object')

In [5]:
df_I10_h = df_I10[['EDC_SBA','OPRTN_YN','VSHSP_DD_CNT','BTH_YYYY']]
df_I10_h  #분석할 컬럼들만 추출

,EDC_SBA,OPRTN_YN,VSHSP_DD_CNT,BTH_YYYY
0,3000,0,1,1921LE
1,6000,0,2,1921LE
2,3000,0,1,1921LE
3,4200,0,1,1921LE
4,1500,0,1,1921LE
...,...,...,...,...
4200,3000,0,1,2005
4201,3000,0,1,2005
4202,3000,0,1,2005
4203,3000,0,1,2005


## 목표변수 검증

Y : 연속형 : 단일 변수에 대한 정규성 여부 판단

    - 검증 갯수 따른 방법 선택 (레코드 갯수)
        - 5천개보다 이하 : shapiro normality test
        - 5천개보다 이상 : Anderson normality test

선정 변수:
    
    - 분석 내용 : 심결본인부담금 (EDC_SBA)가 정규 분포를 따르는가?
        - 귀무가설 : 심결본인부담금 (EDC_SBA)가 정규 분포이다.
        - 대립가설 : 심결본인부담금 (EDC_SBA)가 정규 분포가 아니다.

    - 분석결론 (기준:p-value 0.05) / P.value < 0.05 : 대립 참
        - 통계 결론 : pvalue=0.0이므로 대립가설 참
        - 사실 결론 : 심결본인부담금 (EDC_SBA)가 정규 분포가 아니다.

## 설명변수 검증
- VSHSP_DD_CNT(공상 등 구분) - 연속형
- OPRTN_YN(수술여부) - 범주형 
- BTH_YYYY(출생년도) - 연속형 

In [6]:
df_I10_h = df_I10_h[~df_I10_h["BTH_YYYY"].str.contains("1921LE", case=False)] #LE 있는 데이터 삭제
df_I10_h["BTH_YYYY"] = df_I10_h["BTH_YYYY"].astype(int)

C:\Users\04-04\AppData\Local\Temp\ipykernel_25084\2225468283.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_I10_h["BTH_YYYY"] = df_I10_h["BTH_YYYY"].astype(int)


In [7]:
df_I10_h["OPRTN_YN"] = df_I10_h["OPRTN_YN"].astype(object)

C:\Users\04-04\AppData\Local\Temp\ipykernel_25084\1581453411.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_I10_h["OPRTN_YN"] = df_I10_h["OPRTN_YN"].astype(object)


In [8]:
df_I10_h.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4194 entries, 11 to 4204
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   EDC_SBA       4194 non-null   int64 
 1   OPRTN_YN      4194 non-null   object
 2   VSHSP_DD_CNT  4194 non-null   int64 
 3   BTH_YYYY      4194 non-null   int32 
dtypes: int32(1), int64(2), object(1)
memory usage: 147.4+ KB


#### X : 연속형 / Y : 연속형  = 주로 상관관계 확인에서 사용
    - correlation coefficient(상관계수)
    - 1 ~ 1
    - 0에 가까우면 관계없음, -1로 갈수록 반비례, 1로 갈수로 정비례

In [9]:
df_I10_h.describe().corr()

,EDC_SBA,VSHSP_DD_CNT,BTH_YYYY
EDC_SBA,1.000000,-0.136101,-0.017341
VSHSP_DD_CNT,-0.136101,1.000000,0.793223
BTH_YYYY,-0.017341,0.793223,1.000000


#### 선정 변수: 심결본인부담금(EDC_SBA) , 생년월일(BTH_YYYY)
- 분석 내용 : 출생년도에 따라 요양개시일자에 관계가 있는가?
    - 귀무가설 : EDC_SBA(심결본인부담금),BTH_YYYY(출생년도)에 상관관계가 없다.
    - 대립가설 : EDC_SBA(심결본인부담금),BTH_YYYY(출생년도)에에 상관관계가 있다
- 분석결론 (기준:p-value 0.05) / P.value < 0.05 : 대립 참
    - 통계 결론 : 귀무가설 참,
    - 사실 결론 : 
  
  SignificanceResult(statistic=-0.04424737115085244, pvalue=0.0026338234621164462)

In [12]:
stats.spearmanr(df_I10_h['EDC_SBA'], df_I10_h['BTH_YYYY'])

SignificanceResult(statistic=0.041196604292383456, pvalue=0.0076241866700356795)

#### X : 범주형 / Y : 연속형  = 주로 상관관계 확인에서 사용
 - record 갯수 5천개 이하 : shapiro test (정규성 검증)
 - 정규분포 등분산 검증방법
     -  p_value >  0.05 등분산
     -  p_value <  0.05 이분산

#### 선정 변수: EDC_SBA(심결본인부담금) ,OPRTN_YN(수술여부)

In [13]:
statistic, p_value = stats.shapiro(df_I10_h['OPRTN_YN'])

In [29]:
if p_value >  0.05:
    print("등분산")
else:
    print("이분산")

이분산


#### 선정 변수: EDC_SBA(심결본인부담금) ,VSHSP_DD_CNT(입내원일수)